In [1]:
import numpy as np # MATRIX OPERATIONS
import pandas as pd # EFFICIENT DATA STRUCTURES
import matplotlib.pyplot as plt # GRAPHING AND VISUALIZATIONS
import math # MATHEMATICAL OPERATIONS
import cv2 # IMAGE PROCESSING - OPENCV
from glob import glob # FILE OPERATIONS
import itertools

# KERAS AND SKLEARN MODULES
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,CSVLogger

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from keras.preprocessing import image
import os
import random


Using TensorFlow backend.


In [2]:

import zipfile

In [3]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/dance.zip", 'r')
zip_ref.extractall("/dance_ml")
zip_ref.close()

In [4]:
train = pd.read_csv('/dance_ml/dataset/train.csv')
train.head()

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi


In [5]:
train['target'].value_counts()

mohiniyattam     50
odissi           49
kathakali        47
bharatanatyam    47
kuchipudi        46
sattriya         45
kathak           44
manipuri         36
Name: target, dtype: int64

In [6]:
test = pd.read_csv('/dance_ml/dataset/test.csv')
test.head()

,Image
0,508.jpg
1,246.jpg
2,473.jpg
3,485.jpg
4,128.jpg


In [7]:
Class_map={'manipuri':0,'kathak':1,'sattriya':2,'kuchipudi':3,'kathakali':4,'bharatanatyam':5,'odissi':6,'mohiniyattam':7}
inverse_map={0:'manipuri',1:'kathak',2:'sattriya',3:'kuchipudi',4:'kathakali',5:'bharatanatyam',6:'odissi',7:'mohiniyattam'}




In [8]:
train['target']=train['target'].map(Class_map)

In [9]:
train_img=[]
train_label=[]
j=0
path='/dance_ml/dataset/train'
for i in tqdm(train['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(224,224))
    img=img.astype('float32')
    train_img.append(img)
    train_label.append(train['target'][j])
    j=j+1

100%|██████████| 364/364 [00:02<00:00, 170.10it/s]


In [10]:
test_img=[]
path='/dance_ml/dataset/test'
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(224,224))
    img=img.astype('float32')
    test_img.append(img)

100%|██████████| 156/156 [00:00<00:00, 163.78it/s]


In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagenerator = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        rotation_range = 10,  
        zoom_range = 0.10,  
        width_shift_range=0.10,  
        height_shift_range=0.10,  
        horizontal_flip=True,  
        vertical_flip=False) 


datagenerator.fit(train_img)

In [33]:
train_img=np.array(train_img)
test_img=np.array(test_img)
train_label=np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

(364, 224, 224, 3)
(156, 224, 224, 3)
(364,)


In [34]:

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import *
from keras.models import Sequential
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3),pooling='avg')
vgg16.trainable = False

model = Sequential([
  vgg16, 
  Dense(1024, activation='relu'),
  Dropout(0.5),
  
  
  
 
  Dense(8, activation='softmax'),
])

In [35]:
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
vgg16.trainable=False

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.2,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

callbacks = [reduce_learning_rate]
    


model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(datagenerator.flow(train_img, to_categorical(train_label,8), batch_size=64),
                    epochs=35,steps_per_epoch = 10, callbacks=callbacks)

Epoch 1/35
10/10 [==============================] - 9s 858ms/step - loss: 7.0774 - accuracy: 0.2700
Epoch 2/35
10/10 [==============================] - 8s 840ms/step - loss: 2.8361 - accuracy: 0.5290
Epoch 3/35
10/10 [==============================] - 8s 820ms/step - loss: 1.5265 - accuracy: 0.6850
Epoch 4/35
10/10 [==============================] - 8s 821ms/step - loss: 0.9356 - accuracy: 0.7767
Epoch 5/35
10/10 [==============================] - 8s 836ms/step - loss: 0.5618 - accuracy: 0.8258
Epoch 6/35
10/10 [==============================] - 8s 817ms/step - loss: 0.4895 - accuracy: 0.8383
Epoch 7/35
10/10 [==============================] - 8s 843ms/step - loss: 0.3536 - accuracy: 0.8823
Epoch 8/35
10/10 [==============================] - 8s 815ms/step - loss: 0.3121 - accuracy: 0.9033
Epoch 9/35
10/10 [==============================] - 8s 808ms/step - loss: 0.2213 - accuracy: 0.9267
Epoch 10/35
10/10 [==============================] - 8s 823ms/step - loss: 0.1565 - accuracy: 0.9500

In [36]:
labels = model.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.to_csv('result5.csv', index=False)